# Testing different implementations of randomisation in `Moran_Local`

In [23]:
from importlib import reload
import esda
import libpysal as ps
import geopandas
import pandas
import numpy as np

PERMS = 9999

from libpysal.examples import get_path, load_example

In [31]:
_ = load_example("NCOVR")

nat = geopandas.read_file(get_path("NAT.shp"))

- Load up data

In [30]:
y_name = "HR70"
db = geopandas.read_file(get_path("NAT.shp"))

w = ps.weights.Queen.from_dataframe(db)
cardinalities = pandas.Series(w.cardinalities).values

## Permutation index generator

- Original implementation

In [52]:
nn = w.n - 1
k = cardinalities.max() + 1
prange = list(range(PERMS))

In [54]:
%timeit rids = np.array([np.random.permutation(nn)[0:k] for i in prange])

25.7 ms ± 84.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


- Levi `numba`

In [55]:
rids = esda.moran.vec_permutations(nn, k)
%timeit rids = esda.moran.vec_permutations(nn, k)

91.4 µs ± 3.01 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## LISA conditional randomisation

- Example with "as is" codebase:

In [31]:
lisa = esda.Moran_Local(db[y_name], w, permutations=PERMS)

- Implementation "as is"

In [32]:
%timeit lisa = esda.Moran_Local(db[y_name], w, permutations=PERMS)

9.82 s ± 8.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- Example with `neighbors_perm`

In [33]:
reload(esda)
acc, out = esda.moran.neighbors_perm(lisa.z, 
                                     lisa.Is, 
                                     cardinalities, 
                                     None, 
                                     w.sparse.data,
                                     PERMS,
                                     False
                                    )
%timeit acc, out = esda.moran.neighbors_perm(lisa.z, lisa.Is, cardinalities, None, w.sparse.data, PERMS, False)

27.9 s ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
reload(esda)
acc, out = esda.moran.neighbors_perm_plus(lisa.z, 
                                     lisa.Is, 
                                     cardinalities, 
                                     w.sparse.data,
                                     PERMS,
                                     False
                                    )
%timeit acc, out = esda.moran.neighbors_perm_plus(lisa.z, lisa.Is, cardinalities, w.sparse.data, PERMS, False)

4.03 s ± 6.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
